In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import fbeta_score
from sklearn.preprocessing import LabelEncoder
import re

from imblearn.over_sampling import SMOTE
import numpy as np

In [2]:
data_rare = pd.read_csv('data_rare.csv', index_col=[0])

In [3]:
data_rare.loc[~data_rare['handshake'].isna(), 'is_handshake'] = 1
data_rare.loc[data_rare['is_handshake'].isna(), 'is_handshake'] = 0

In [6]:
fingerprint = []
alert_message = []
certificate = []
is_selfsigned = []

for i in range(0, len(data_rare)):
    fingerprint.append(len(re.findall('fingerprint', data_rare['handshake'].fillna('0')[i], flags=0)))
    alert_message.append(len(re.findall('alert_message', data_rare['packets'].fillna('0')[i], flags=0)))
    certificate.append(len(re.findall('certificate', data_rare['handshake'].fillna('0')[i], flags=0)))
    is_selfsigned.append(len(re.findall("'is_selfsigned': True", data_rare['handshake'].fillna('0')[i], flags=0)))

data_rare['fingerprint'] = fingerprint
data_rare['alert_message'] = alert_message
data_rare['certificate'] = certificate
data_rare['is_selfsigned'] = is_selfsigned

In [7]:
data_rare['country'] = pd.read_csv('data.csv')['country']

In [8]:
data_rare = data_rare.drop('index', axis=1)

In [9]:
X = data_rare.drop(['packets', 'timestamps', 'anomaly', 'handshake'], axis=1)

In [10]:
X['addr_1'] = pd.DataFrame(list(X['addr'].str.split('.').values), columns = ['addr_1',
                                                        'addr_2',
                                                        'addr_3', 
                                                        'addr_4'])['addr_1']

X['addr_2'] = pd.DataFrame(list(X['addr'].str.split('.').values), columns = ['addr_1',
                                                        'addr_2',
                                                        'addr_3', 
                                                        'addr_4'])['addr_2']

In [11]:
X_new = X.drop(['proto', 'addr_1', 'addr_2', 'country', 'addr'], 
                     axis=1).merge(pd.get_dummies(X[['proto', 'country', 'addr_1', 'addr_2']], drop_first = True),
                                                    left_index = True, right_index=True)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, data_rare['anomaly'], random_state = 42, test_size = 0.3)

In [14]:
# Нормализация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Преобразование данных в тензоры
X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train.values, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test.values, dtype=torch.float)

### 1 подход

In [22]:
# Определение нейронной сети
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Параметры
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 10

# Создание модели, оптимизатора и функции потерь
model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train.unsqueeze(1).type_as(outputs))
    loss.backward()
    optimizer.step()

# Оценка модели с помощью Fbeta_score
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    preds = (test_outputs > 0).int()
    targets = y_test.int()

# Оценка с использованием Fbeta_score
fbeta = fbeta_score(targets, preds, beta=1.5)

print("Fbeta_score on test data:", fbeta)

Fbeta_score on test data: 0.0


Странно получилось, посмотрим вообще на все предсказания.

In [25]:
preds.unique()

tensor([0], dtype=torch.int32)

Причина в том, что на наших данных нейронка выдает константный прогноз = 0

### 2 подход
Возможно, причина в дизбалансе классов. У меня данных с аномалиями всего 5%. 
Используем параметр pos_weight в функции потерь nn.BCEWithLogitsLoss, чтобы учитывать дисбаланс классов. Установив вес для класса 1 равным 19, мы помогаем модели более эффективно обучаться на дисбалансированных данных.

In [31]:
# Определение нейронной сети
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Параметры
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 10

# Создание модели, оптимизатора и взвешенной функции потерь
model = NeuralNetwork(input_size, hidden_size, output_size)
pos_weight = torch.tensor([19.0])  # Устанавливаем вес для класса 1
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train.unsqueeze(1).type_as(outputs))
    loss.backward()
    optimizer.step()

# Оценка модели с помощью Fbeta_score
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    preds = (test_outputs > 0).int()
    targets = y_test.int()

# Оценка с использованием Fbeta_score
fbeta = fbeta_score(targets, preds, beta=1.5)

print("Fbeta_score on test data:", fbeta)

Fbeta_score on test data: 0.0


In [33]:
preds.unique()

tensor([0], dtype=torch.int32)

Тоже не сработало.

### Попробуем ребалансировку классов
Просто использовали алгоритм SMOTE для ресемплирования данных и балансировки классов перед обучением модели.

In [43]:
# Создание экземпляра SMOTE
sm = SMOTE(random_state=42)

# Ресемплирование тренировочных данных
X_train_resampled, y_train_resampled = sm.fit_resample(np.nan_to_num(X_train), np.nan_to_num(y_train))

# Преобразование в тензоры PyTorch
X_train_resampled = torch.FloatTensor(X_train_resampled)
y_train_resampled = torch.FloatTensor(y_train_resampled)

# Создание модели, оптимизатора и критерия потерь
model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели на ресемплированных данных
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_resampled)
    loss = criterion(outputs, y_train_resampled.unsqueeze(1))
    loss.backward()
    optimizer.step()

# Оценка модели с помощью Fbeta_score
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    preds = (test_outputs > 0).int()
    targets = y_test.int()

# Оценка с использованием Fbeta_score
fbeta = fbeta_score(targets, preds, beta=1.5)

print("Fbeta_score on test data:", fbeta)

Fbeta_score on test data: 0.33455299426759366


In [45]:
preds.unique()

tensor([0, 1], dtype=torch.int32)

Кажется, это лучше, но все еще качество недотягивает до нашей финальной ml модели